# Multiple Species Tree visualizations
In this tutorial, we will visualize the Species Tree graph using all of the available embedding methods that can embed it.

This will exclude methods that require edge weights.

## Retrieving the graph

In [1]:
from grape.datasets.string import SpeciesTree
graph = SpeciesTree()

To get the graph report, just run:

In [2]:
graph

<div class="graph-report"><style>.graph-report li {margin: 0.5em 0 0.5em 0;}.graph-report .paragraph {text-align: justify;word-break: break-all;}.graph-report .small-columns {column-count: 4;column-gap: 2em;}.graph-report .medium-columns {column-count: 3;column-gap: 2em;}.graph-report .large-columns {column-count: 2;column-gap: 2em;}.graph-report .single-column {}@media only screen and (max-width: 600px) {.graph-report .small-columns {column-count: 1;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 600px) and (max-width: 800px) {.graph-report .small-columns {column-count: 2;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 800px) and (max-width: 1024px) {.graph-report .small-columns {column-count: 3;}.graph-report .medium-columns {column-count: 2;}.graph-report .large-columns {column-count: 1;}}</style><h2>SpeciesTree</h2><p class="paragraph">The undirected graph SpeciesTree has 24.62K heterogeneous nodes and 24.61K heterogeneous edges. The graph is connected, that is, it is composed of a single connected component that includes all nodes and edges. The RAM requirements for the nodes and edges data structures are 3.63MB and 99.70KB respectively.</p><h3 style="margin: 1em 0 0 0;">Degree centrality</h3><p class="paragraph">The minimum node degree is 1, the maximum node degree is 163, the mode degree is 1, the mean degree is 2.00 and the node degree median is 1.</p><p class="paragraph">The nodes with the highest degree centrality are Lactobacillus (degree 163 and node type Bacteria), Streptomyces (degree 156 and node type Bacteria), Bacillus (degree 128 and node type Bacteria), unclassified Streptomyces (degree 123 and node type Bacteria) and Rhodobacteraceae (degree 105 and node type Bacteria).</p><h3 style="margin: 1em 0 0 0;">Node types</h3><p class="paragraph">The graph has 4 node types, which are Bacteria (19.13K nodes, 77.72%), Eukaryota (4.50K nodes, 18.27%), Archaea (987 nodes, 4.01%) and Ancestral. The RAM requirement for the node types data structure is 1.28MB.</p><h4 style="margin: 1em 0 0 0;">Singleton node types</h4><p class="paragraph">Singleton node types are node types that are assigned exclusively to a single node, making the node type relatively meaningless, as it adds no more information than the node name itself. The graph contains a singleton node type, which is Ancestral (node cellular organisms (degree 3 and node type Ancestral)).</p><h3 style="margin: 1em 0 0 0;">Edge types</h3><p class="paragraph">The graph has 3 edge types, which are Bacteria (38.26K edges, 77.72%), Eukaryota (8.99K edges, 18.27%) and Archaea (1.97K edges, 4.01%). The RAM requirement for the edge types data structure is 197.30KB.</p><h3 style="margin: 1em 0 0 0;">Topological Oddities</h3><p class="paragraph">A topological oddity is a set of nodes in the graph that <i>may be derived</i> by an error during the generation of the edge list of the graph and, depending on the task, could bias the results of topology-based models. In the following paragraph, we will describe the detected topological oddities.</p><h4 style="margin: 1em 0 0 0;">Trees</h4><p class="paragraph">A tree is a connected component with <code>n</code> nodes and <code>n-1</code> edges. We have detected  a single tree in the graph, involving a total of 24.62K nodes (100.00%) and 24.61K edges (50.00%).</p><ul class="large-columns"><li><p class="paragraph">Tree starting from the root node Chordata (degree 4 and node type Eukaryota), and containing 24.62K nodes, with a maximal depth of 32, which are Tunicata (node type Eukaryota), Cephalochordata (node type Eukaryota), Deuterostomia (degree 4 and node type Eukaryota), Craniata (node type Eukaryota) and Ascidiacea (node type Eukaryota). Its nodes have 4 node types, which are Bacteria (19.13K nodes, 77.72%), Eukaryota (4.50K nodes, 18.26%), Archaea (987 nodes, 4.01%)

## Sorting the graph by decreasing node degree
Some of the TensorFlow embedding models use the TensorFlow NCE method, which requires the nodes to be sorted by occurrence.

In [3]:
graph = graph.sort_by_decreasing_outbound_node_degree()

## Splitting into training and test set

In [4]:
train, test = graph.get_node_label_holdout_graphs(train_size=0.7)
train.enable()

## Retrieving the available embedding methods

In [5]:
from grape import get_available_models_for_node_embedding

all_embedding_methods = get_available_models_for_node_embedding()
embedding_methods = all_embedding_methods[~all_embedding_methods.requires_edge_weights.astype(bool)]

# We remove some models that either require some additional
# information or are simply currently work in progress.
embedding_methods = embedding_methods[
    ~embedding_methods.model_name.str.contains("SPINE") &
    ~embedding_methods.model_name.str.contains("WINE")
]

## Running the visualizations

In [ ]:
import silence_tensorflow.auto
from grape import GraphVisualizer
from grape.utils import AbstractEmbeddingModel
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from math import ceil

ncols = 6
nrows = ceil(embedding_methods.shape[0] / ncols)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(6*ncols, 6*nrows))

for index, (_, row) in tqdm(
    enumerate(embedding_methods.iterrows()),
    total=embedding_methods.shape[0],
    desc="Computing embedding"
):
    model = AbstractEmbeddingModel.get_model_from_library(
        model_name=row.model_name,
        library_name=row.library_name
    )(enable_cache=True)
    embedding = model.fit_transform(train)
    vis = GraphVisualizer(
        graph,
        node_embedding_method_name=f"{row.model_name} {row.library_name}",
        automatically_display_on_notebooks=False
    )
    vis.fit_nodes(embedding)
    vis.plot_node_types(
        figure=fig,
        axes=axes[index // ncols, index % ncols],
        train_indices=train.get_node_ids_with_known_node_types(),
        test_indices=test.get_node_ids_with_known_node_types()
    )
    
for axis in axes.flatten()[index:]:
    for spine in axis.spines.values():
        spine.set_visible(False)
    axis.axis("off")
    
fig.tight_layout()
plt.show()

Computing embedding:   0%|          | 0/65 [00:00<?, ?it/s]